In [1]:
!pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 660.6 kB/s eta 0:00:02
   - -------------------------------------- 0.0/1.2 MB 660.6 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.2 MB 363.1 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.2 MB 476.3 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.2 MB 435.7 kB/s eta 0:00:03
   ----- ---------------------------------- 0.2/1.2 MB 546.6 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.2 MB 546.6 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.2 MB 455.1 kB/s eta 0:00:03
   ---------- ----------------------------- 0.3/1.2 MB 679.4 kB/s eta 0:00:02
   ---------- ----------------------------- 0.3/1.2 MB 633.2 kB/s eta 0:00:02
   ---------------- ----------------------- 0.5/1.2 MB 867.8 kB/s eta 0:00:01
   -------------------- ------------------- 0.6/1.2 MB 992.1 kB/s eta 0:0

In [2]:
import pandas as pd
from sqlalchemy import create_engine

DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "postgres1"
DB_HOST = "172.17.5.39"
DB_PORT = "5432"
SOURCE_SCHEMA_NAME = "DP_OLTP_HEALTHCARE"
TARGET_SCHEMA_NAME = "DP_OLAP_HEALTHCARE"

# Database connection parameters
source_engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
target_engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")



In [10]:
ClinicLocations

,clinicnbr,displayname,address1,address2,city,state,zip,region
0,785522,Evergreen Clinic,"Smith, Woodard and Sweeney Building",99924 James Trail,Columbus,Ohio,43109,Midwest
1,785623,Twin Mountains Hospital,King Ltd Building,619 Bowen Springs,Jacksonville,Florida,32224,South
2,785724,Big Heart Community Hospital,"Yoder, Smith and Poole Building",7896 Kimberly Centers,Frankfort,Illinois,40601,Midwest
3,785825,Pioneer Clinic,Serrano-Reyes Building,32337 Lori Crossroad,Auburn,New York,36849,Northeast
4,785926,Fairmont Hospital Center,Gordon-Scott Building,102 Clarence Estates,Beaumont,Texas,77708,South
5,786027,Angelstone Community Hospital,Tate Group Building,131 Patricia Island,Austin,Texas,78749,South
6,786128,Genesis Hospital Center,Dyer PLC Building,70270 Dominguez Dam,Marysville,Washington,98271,West
7,786229,Principal Medical Clinic,Gamble-Nguyen Building,92633 Martinez Court,Ann Arbor,Michigan,48103,Midwest
8,786330,Fairview General Hospital,"Massey, Wright and West Building",561 Melissa Mountains,Dublin,California,94568,West
9,786431,Guardian Medical Clinic,Palmer Inc Building,842 Sarah Haven,Bridgeton,New Jersey,63044,Northeast


In [15]:
# Read data from Billing, ClinicLocations and ClinicalReports table
Billing = pd.read_sql(f'SELECT * FROM "{SOURCE_SCHEMA_NAME}".Billing', source_engine)
ClinicLocations = pd.read_sql(f'SELECT * FROM "{SOURCE_SCHEMA_NAME}".ClinicLocations', source_engine)
ClinicalReports = pd.read_sql(f'SELECT * FROM "{SOURCE_SCHEMA_NAME}".ClinicalReports', source_engine)
FactTransactions = pd.merge(Billing, ClinicLocations, left_on='facility', right_on='displayname', how='left')


# Write transformed data to FactTransactions table
# FactTransactions.to_sql('dimpatient', target_engine, schema=TARGET_SCHEMA_NAME, if_exists='append', index=False)

In [16]:
FactTransactions

,billnbr,billdate,patientnbr,providernbr,facility,diagnosisreportref,servicereportref,billamount,insurancecoverage,paymentrecieved,clinicnbr,displayname,address1,address2,city,state,zip,region
0,897144565,2021-12-18,5500208,215189,Angelstone Community Hospital,DIAG-897144565,SERV-897144565,26.91,16.0,10.91,786027,Angelstone Community Hospital,Tate Group Building,131 Patricia Island,Austin,Texas,78749,South
1,897144566,2021-12-18,5500208,215189,Angelstone Community Hospital,DIAG-897144566,SERV-897144566,35.88,24.0,11.88,786027,Angelstone Community Hospital,Tate Group Building,131 Patricia Island,Austin,Texas,78749,South
2,897145888,2021-12-18,5500208,216456,Angelstone Community Hospital,DIAG-897145888,SERV-897145888,52.20,38.0,14.20,786027,Angelstone Community Hospital,Tate Group Building,131 Patricia Island,Austin,Texas,78749,South
3,897145889,2021-12-18,5500208,216456,Angelstone Community Hospital,DIAG-897145889,SERV-897145889,52.20,37.0,15.20,786027,Angelstone Community Hospital,Tate Group Building,131 Patricia Island,Austin,Texas,78749,South
4,897162391,2021-12-18,5500208,216456,Angelstone Community Hospital,DIAG-897162391,SERV-897162391,42.18,28.0,14.18,786027,Angelstone Community Hospital,Tate Group Building,131 Patricia Island,Austin,Texas,78749,South
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83994,1129229709,2022-05-28,5502139,216427,Angelstone Community Hospital,DIAG-1129229709,SERV-1129229709,155.16,0.0,155.16,786027,Angelstone Community Hospital,Tate Group Building,131 Patricia Island,Austin,Texas,78749,South
83995,1129229710,2022-06-27,5502139,216427,Angelstone Community Hospital,DIAG-1129229710,SERV-1129229710,620.62,0.0,620.62,786027,Angelstone Community Hospital,Tate Group Building,131 Patricia Island,Austin,Texas,78749,South
83996,1129229711,2022-07-27,5502139,216427,Angelstone Community Hospital,DIAG-1129229711,SERV-1129229711,310.31,0.0,310.31,786027,Angelstone Community Hospital,Tate Group Building,131 Patricia Island,Austin,Texas,78749,South
83997,1129229712,2022-04-30,5502139,216427,Angelstone Community Hospital,DIAG-1129229712,SERV-1129229712,620.62,566.0,54.62,786027,Angelstone Community Hospital,Tate Group Building,131 Patricia Island,Austin,Texas,78749,South


In [36]:
import pandas as pd
from sqlalchemy import create_engine

DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "postgres1"
DB_HOST = "172.17.5.39"
DB_PORT = "5432"
SOURCE_SCHEMA_NAME = "dp_oltp_healthcare"
TARGET_SCHEMA_NAME = "dp_olap_healthcare"

# Database connection parameters
source_engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
target_engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")


# Function to load data from source table and transform for FactTransactions
def load_transform_fact_transactions():
    # Read data from billing table
    billing_df = pd.read_sql(f'SELECT * FROM {SOURCE_SCHEMA_NAME}.billing', source_engine)
    
    # Read data from cliniclocations table
    clinic_locations_df = pd.read_sql(f'SELECT * FROM {SOURCE_SCHEMA_NAME}.cliniclocations', source_engine)
    
    # Read data from clinicalreports table
    clinical_reports_df1 = pd.read_sql(f'SELECT DiagnosisCode, ReportRefNbr FROM {SOURCE_SCHEMA_NAME}.clinicalreports', source_engine)
    
    # Read data from clinicalreports table
    clinical_reports_df2 = pd.read_sql(f'SELECT CPTCode, GeneratedDate, CPTUnits, ReportRefNbr  FROM {SOURCE_SCHEMA_NAME}.clinicalreports', source_engine)
    
    # Merge dataframes to perform joins
    merged_df = pd.merge(billing_df, clinic_locations_df, how='left', left_on='facility', right_on='displayname')
    merged_df = pd.merge(merged_df, clinical_reports_df1, how='left', left_on='diagnosisreportref', right_on='reportrefnbr', suffixes=('', '_diagnosis'))
    merged_df = pd.merge(merged_df, clinical_reports_df2, how='left', left_on='servicereportref', right_on='reportrefnbr', suffixes=('', '_service'))
    
    # Rename dictionary to store original column names and renamed column names
    rename_dict = {'billnbr': 'transactionid',
                   'patientnbr': 'patientid',
                   'providernbr': 'physicianid',
                   'clinicnbr': 'hospitalid',
                   'diagnosiscode': 'diagnosiscodeid',
                   'cptcode': 'cptcodeid',
                   'generateddate': 'servicedate',
                   'billdate': 'paymentdate',
                   'cptunits': 'cptunits',
                   'billamount': 'billedamount',
                   'insurancecoverage': 'insurancepayment',
                   'paymentrecieved': 'patientpayment'}
    
    # Select only the columns that are being renamed
    selected_columns = list(rename_dict.keys())
    
    # Rename columns in the merged DataFrame
    transformed_df = merged_df[selected_columns].rename(columns=rename_dict)
    
    # Write transformed data to target table
    transformed_df.to_sql('facttransactions', target_engine, schema=TARGET_SCHEMA_NAME, if_exists='append', index=False)

# Function to load data from source table and transform for dimpatient
def load_transform_dim_patient():
    # Read data from patient and patientsurvey tables
    patient_df = pd.read_sql(f'SELECT * FROM {SOURCE_SCHEMA_NAME}.patient', source_engine)
    patient_survey_df = pd.read_sql(f'SELECT * FROM {SOURCE_SCHEMA_NAME}.patientsurvey', source_engine)
    
    # Merge dataframes to perform join
    merged_df = pd.merge(patient_df, patient_survey_df, how='left', left_on='patientnbr', right_on='surveynbr')
    
    # Rename dictionary to store original column names and renamed column names
    rename_dict = {'patientnbr': 'patientid',
                   'registrationdate': 'registrationdate',
                   'healthcardnbr': 'healthcardnumber',
                   'firstname': 'firstname',
                   'lastname': 'lastname',
                   'email': 'email',
                   'gender': 'gender',
                   'dob': 'dob',
                   'heightcms': 'heightcms',
                   'heightin': 'heightin',
                   'weightlbs': 'weightlbs',
                   'weightkgs': 'weightkgs',
                   'bloodgroup': 'bloodgroup',
                   'tobaccouser': 'tobacco',
                   'alcoholuser': 'alcohol',
                   'exercisefrequency': 'exercise',
                   'ondiet': 'diet',
                   'ethnicity': 'ethinicity',
                   'address1': 'address1',
                   'address2': 'address2',
                   'city': 'city',
                   'state': 'state',
                   'statecode': 'statecode',
                   'zip': 'zipcode',
                   'region': 'region'}
    
    # Select only the columns that are being renamed
    selected_columns = list(rename_dict.keys())
    
    # Rename columns in the merged DataFrame
    transformed_df = merged_df[selected_columns].rename(columns=rename_dict)
    
    # Write transformed data to target table
    transformed_df.to_sql('dimpatient', target_engine, schema=TARGET_SCHEMA_NAME, if_exists='append', index=False)

    
# Function to load data from source table and transform for dimphysician
def load_transform_dim_physician():
    # Read data from provider and providerspecialty tables
    provider_df = pd.read_sql(f'SELECT * FROM {SOURCE_SCHEMA_NAME}.provider', source_engine)
    provider_specialty_df = pd.read_sql(f'SELECT * FROM {SOURCE_SCHEMA_NAME}.providerspecialty', source_engine)
    
    # Merge dataframes to perform join
    merged_df = pd.merge(provider_df, provider_specialty_df, how='left', left_on='specialitycode', right_on='specialitycode')
    
    # Rename dictionary to store original column names and renamed column names
    rename_dict = {'providernbr': 'providerid',
                   'npinbr': 'npi',
                   'prefix': 'prefix',
                   'firstname': 'firstname',
                   'lastname': 'lastname',
                   'email': 'email',
                   'fte': 'fte',
                   'providercategory_x': 'providertype',
                   'specialitytype': 'specialitytype'}
    
    # Select only the columns that are being renamed
    selected_columns = list(rename_dict.keys())

    # Rename columns in the merged DataFrame
    transformed_df = merged_df[selected_columns].rename(columns=rename_dict)

    # Write transformed data to target table
    transformed_df.to_sql('dimphysician', target_engine, schema=TARGET_SCHEMA_NAME, if_exists='append', index=False)

# Function to load data from source table and transform for dimhospital
def load_transform_dim_hospital():
    # Read data from cliniclocations table
    clinic_locations_df = pd.read_sql(f'SELECT * FROM {SOURCE_SCHEMA_NAME}.cliniclocations', source_engine)
    
    # Rename dictionary to store original column names and renamed column names
    rename_dict = {'clinicnbr': 'hospitalid',
                   'displayname': 'hospitalname',
                   'address1': 'address1',
                   'address2': 'address2',
                   'city': 'city',
                   'state': 'state',
                   'zip': 'zipcode',
                   'region': 'region'}
    
    # Select only the columns that are being renamed
    selected_columns = list(rename_dict.keys())

    # Rename columns in the DataFrame
    transformed_df = clinic_locations_df[selected_columns].rename(columns=rename_dict)
    
    # Write transformed data to target table
    transformed_df.to_sql('dimhospital', target_engine, schema=TARGET_SCHEMA_NAME, if_exists='append', index=False)

# Function to load data from source table and transform for dimdiagnosiscode
def load_transform_dim_diagnosis_code():
    # Read data from diagnosiscodelookup table
    diagnosis_code_lookup_df = pd.read_sql(f'SELECT * FROM {SOURCE_SCHEMA_NAME}.diagnosiscodelookup', source_engine)
    
    # Rename dictionary to store original column names and renamed column names
    rename_dict = {'diagnosiscode': 'diagnosiscode',
                   'diagnosisgrouping': 'diagnosiscodedescription',
                   'diagnosisdescription': 'diagnosiscodegroup'}
    
    # Select only the columns that are being renamed
    selected_columns = list(rename_dict.keys())

    # Rename columns in the DataFrame
    transformed_df = diagnosis_code_lookup_df[selected_columns].rename(columns=rename_dict)
    
    # Write transformed data to target table
    transformed_df.to_sql('dimdiagnosiscode', target_engine, schema=TARGET_SCHEMA_NAME, if_exists='append', index=False)

    
# Function to load data from source table and transform for dimcptcode
def load_transform_dim_cpt_code():
    # Read data from cptcodeslookup table
    cpt_codes_lookup_df = pd.read_sql(f'SELECT * FROM {SOURCE_SCHEMA_NAME}.cptcodeslookup', source_engine)
    
    # Rename dictionary to store original column names and renamed column names
    rename_dict = {'cptcode': 'cptcode',
                   'cptgrouping': 'cptcodedescription',
                   'cptdescription': 'cptcodegroup'}
    
    # Select only the columns that are being renamed
    selected_columns = list(rename_dict.keys())

    # Rename columns in the DataFrame
    transformed_df = cpt_codes_lookup_df[selected_columns].rename(columns=rename_dict)
    
    # Write transformed data to target table
    transformed_df.to_sql('dimcptcode', target_engine, schema=TARGET_SCHEMA_NAME, if_exists='append', index=False)


In [37]:
# Load and transform data for each table
def main():
    load_transform_fact_transactions()
    print('load_transform_fact_transactions')
    load_transform_dim_patient()
    print('load_transformh_dim_patient')
    load_transform_dim_physician()
    print('load_transform_dim_physician')
    load_transform_dim_hospital()
    print('load_transform_dim_hospital')
    load_transform_dim_diagnosis_code()
    print('load_transform_dim_diagnosis_code')
    load_transform_dim_cpt_code()
    print('load_transform_dim_cpt_code')

if __name__ == "__main__":
    main()

load_transform_fact_transactions
load_transformh_dim_patient
load_transform_dim_physician
load_transform_dim_hospital
load_transform_dim_diagnosis_code
load_transform_dim_cpt_code


In [4]:
from sqlalchemy import create_engine
import pandas as pd

# Database connection parameters
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "postgres1"
DB_HOST = "172.17.5.39"
DB_PORT = "5432"
SOURCE_SCHEMA_NAME = "DP_OLTP_HEALTHCARE"
TARGET_SCHEMA_NAME = "DP_OLAP_HEALTHCARE"

# Create a database connection engine
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")


In [6]:
# Read source tables from OLTP DB into a pandas DataFrame
Billing = pd.read_sql(f'SELECT * FROM "{SOURCE_SCHEMA_NAME}"."Billing"', engine)
ClinicLocations = pd.read_sql(f'SELECT * FROM "{SOURCE_SCHEMA_NAME}"."ClinicLocations"', engine)
ClinicalReports = pd.read_sql(f'SELECT * FROM "{SOURCE_SCHEMA_NAME}"."ClinicalReports"', engine)

# Convert columns to datetime
Billing['BillDate'] = pd.to_datetime(Billing['BillDate'])
ClinicalReports['GeneratedDate'] = pd.to_datetime(ClinicalReports['GeneratedDate'])

# Filter only the Incremental Data
# Billing = Billing[Billing['BillDate'] > last_processed_date]
# ClinicalReports = ClinicalReports[(ClinicalReports['GeneratedDate'] > last_processed_date) & (ClinicalReports['GeneratedDate'] <= max_processed_date)]

# Join tables to form the FactTransactions Table
FactTransactions = pd.merge(Billing, ClinicLocations[['DisplayName', 'ClinicNbr']], how='left', left_on='Facility', right_on='DisplayName')
FactTransactions = pd.merge(FactTransactions, ClinicalReports[['ReportRefNbr', 'DiagnosisCode']], how='left', left_on='DiagnosisReportRef', right_on='ReportRefNbr')
FactTransactions = pd.merge(FactTransactions, ClinicalReports[['ReportRefNbr', 'CPTCode', 'GeneratedDate', 'CPTUnits']], how='left', left_on='ServiceReportRef', right_on='ReportRefNbr')

# Rename Columns
rename_dict = {
    'BillNbr': 'TransactionID',
    'PatientNbr': 'PatientID',
    'ProviderNbr': 'PhysicianID',
    'ClinicNbr': 'HospitalID',
    'DiagnosisCode': 'DiagnosisCodeID',
    'CPTCode': 'CPTCodeID',
    'GeneratedDate': 'ServiceDate',
    'BillDate': 'PaymentDate',
    'CPTUnits': 'CPTUnits',
    'BillAmount': 'BilledAmount',
    'InsuranceCoverage': 'InsurancePayment',
    'PaymentRecieved': 'PatientPayment'
}

# # Select only the columns that are being renamed
# selected_columns = list(rename_dict.keys())

# # Rename columns in the merged DataFrame
# FactTransactions = FactTransactions[selected_columns].rename(columns=rename_dict)


In [7]:
FactTransactions

,TransactionID,PatientID,PhysicianID,HospitalID,DiagnosisCodeID,CPTCodeID,ServiceDate,PaymentDate,CPTUnits,BilledAmount,InsurancePayment,PatientPayment
0,897144565,5500208,215189,NaN,NaN,NaN,NaT,2022-01-18,NaN,26.91,16.0,10.91
1,897144566,5500208,215189,NaN,NaN,NaN,NaT,2022-01-18,NaN,35.88,24.0,11.88
2,897145888,5500208,216456,NaN,NaN,NaN,NaT,2022-01-18,NaN,52.20,38.0,14.20
3,897145889,5500208,216456,NaN,NaN,NaN,NaT,2022-01-18,NaN,52.20,37.0,15.20
4,897162391,5500208,216456,NaN,NaN,NaN,NaT,2022-01-18,NaN,42.18,28.0,14.18
...,...,...,...,...,...,...,...,...,...,...,...,...
65662,1129070469,5586714,216126,NaN,N300,3078F,2022-04-28,2022-04-29,4.0,81.20,0.0,81.20
65663,1129194758,5648503,216274,NaN,J960,99232,2022-04-29,2022-04-30,4.0,488.61,391.0,97.61
65664,1129203857,5741272,216158,NaN,P072,99472,2022-04-29,2022-04-30,6.0,908.37,878.0,30.37
65665,1129215662,5648503,215908,NaN,G934,99233,2022-04-29,2022-04-30,2.0,170.75,141.0,29.75
